ETABS DESIGN PIERS AS COLUMNS

Introduction:
- Tool is to design walls as columns to AS3600:2018 Section 10 for ULS load combinations
- ETABS model should be correct and run without errors
- Pier labels should be assigned to walls manually
- Piers assigned should be straight walls only, no T or L shaped walls are permitted
- Two walls meeting at a corner or junction should be assigned two separate pier labels
- Program will design the minimum bar size for the bar spacing selected in each direction
- This may not necessarily comply with code minimum requirements. This is to be checked manually
- X-X is defined as strong direction and Y-Y is to be the weak direction

Load combinations:
- ETABS load combinations must be set up as follows:
- eq_env_1 -> Envelope of load combinations with unfactored earthquake load for flexural design
- eq_env_2 -> Envelope of load combinations with factored earthquake load for shear design
- wind_env -> Envelope of all ULS wind combinations
- If designing non-ductile walls for earthquake actions, eq_env_1 and eq_env_2 will be the same

In [1]:
import sys
sys.path.append(r'C:\_Github\structural_engineering_toolbox')
from etabs_tools import etabs_api, etabs_design
from design_reports import design_reports
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets.widgets import Layout

# Define buttons
connect_button = widgets.Button(description='Connect to ETABS')
design_button = widgets.Button(description='Design piers')
export_button = widgets.Button(description='Export to Excel')

# Create output widgets
output_1 = widgets.Output()  # For dropdowns and other controls
output_2 = widgets.Output()  # For results

# Dictionary for dropdowns
dropdowns = {}

# Initialize designed_piers_df as a global variable
designed_piers_df = None

# Initialize ETABS API
etabs_api_instance = etabs_api.etabs_api()

# Grab levels and determine plastic hinge region levels


# Callback for connect_button
def on_connect_button_clicked(b):
    with output_1:
        clear_output()

        try:
            load_case_list = etabs_api_instance.get_load_cases_and_combinations()
        except Exception as e:
            print(f"Error fetching load cases: {e}")
            return

        dropdowns["eq_env_1"] = widgets.Dropdown(options=load_case_list, description='Unfactored EQ Envelope')
        dropdowns["eq_env_2"] = widgets.Dropdown(options=load_case_list, description='Factored EQ Envelope')
        dropdowns["wind_env"] = widgets.Dropdown(options=load_case_list, description='Wind Envelope')

        spacing_options = [150, 200, 250, 300]
        dropdowns["vertical_spacing"] = widgets.Dropdown(options=spacing_options, value=200, description='Vertical Spacing')
        dropdowns["horizontal_spacing"] = widgets.Dropdown(options=spacing_options, value=200, description='Horizontal Spacing')

        dropdowns['out_of_plane'] = widgets.Checkbox(value=False, description='Design out of plane bending')

        # Arrange widgets in a VBox
        controls_vbox = widgets.VBox([
            dropdowns["eq_env_1"],
            dropdowns["eq_env_2"],
            dropdowns["wind_env"],
            dropdowns["vertical_spacing"],
            dropdowns["horizontal_spacing"],
            dropdowns['out_of_plane'],
            design_button
        ])
        display(controls_vbox)

    with output_2:
        clear_output()

connect_button.on_click(on_connect_button_clicked)

status_label = widgets.Label(value="")

# Callback for design_button
def on_design_button_clicked(b):
    global designed_piers_df  # Declare designed_piers_df as global

    with output_2:
        clear_output()
        status_label.value = "Processing... Please wait."
        display(status_label)

        if not dropdowns:
            print("Error: Please connect to ETABS first.")
            return

        try:
            piers = etabs_api_instance.get_piers(load_cases=[
                dropdowns['eq_env_1'].value,
                dropdowns['eq_env_2'].value,
                dropdowns['wind_env'].value
            ])

        except Exception as e:
            print(f"Error fetching pier data: {e}")
            return

        try:
            designed_piers_df = etabs_design.design_all_piers(
                piers=piers,
                eq_env_1=dropdowns['eq_env_1'].value,
                eq_env_2=dropdowns['eq_env_2'].value,
                wind_env=dropdowns["wind_env"].value,
                vertical_spacing=dropdowns['vertical_spacing'].value,
                horizontal_spacing=dropdowns['horizontal_spacing'].value,
                design_both_axes=dropdowns['out_of_plane'].value
            )

            status_label.value = "Design complete"
            display(export_button, designed_piers_df)

        except Exception as e:
            print(f"Error in design function: {e}")
            status_label.value = ""

design_button.on_click(on_design_button_clicked)

# Callback for export_button
def on_export_button_clicked(b):
    global designed_piers_df  # Declare designed_piers_df as global

    if designed_piers_df is None:
        print("Error: No data to export. Please run the design first.")
        return

    try:
        design_reports.dataframe_to_xlsx(designed_piers_df)
        print("Export successful!")
    except Exception as e:
        print(f"Error during export: {e}")

export_button.on_click(on_export_button_clicked)

# Create the two-column layout using HBox
left_column = widgets.VBox([connect_button, output_1], layout=Layout(width='25%'))  # Left column 25%
right_column = widgets.VBox([output_2], layout=Layout(width='75%'))  # Right column 75%

hbox = widgets.HBox([left_column, right_column])

display(hbox)  # Display the HBox containing both columns

,Pier,Story,Lw (mm),tw (mm),Hw (mm),fc (MPa),V* (kN),Phi Vu,H Bar Size,Ast addl (mm2),Safety Factor Shear,P* Tension (kN),P* Compression (kN),V Bar Size,M* Top X-X (kNm),M* Bot X-X (kNm),Phi Mu X-X,Phi Nu X-X,Safety Factor X-X
0,P01,LEVEL 9,5500.0,300.0,3000.0,50.0,960.0,2822.0,12,3306.0,0.34,-173.0,383.0,12,727.0,83.0,4196.0,-1001.0,0.17
1,P01,LEVEL 8,5500.0,300.0,3000.0,50.0,1224.0,2822.0,12,4794.0,0.43,-624.0,794.0,12,1399.0,620.0,3141.0,-1402.0,0.45
2,P01,LEVEL 7,5500.0,300.0,3000.0,50.0,829.0,2822.0,12,4955.0,0.29,-637.0,1491.0,12,1423.0,1393.0,3135.0,-1404.0,0.45
3,P01,LEVEL 6,5500.0,300.0,4200.0,50.0,1640.0,2822.0,12,6901.0,0.58,-1012.0,2407.0,12,1600.0,838.0,2563.0,-1622.0,0.62
4,P01,LEVEL 5,5500.0,300.0,4200.0,50.0,2037.0,2822.0,12,7876.0,0.72,-1608.0,3617.0,12,1181.0,1050.0,1491.0,-2029.0,0.79
5,P01,LEVEL 4,5500.0,350.0,4200.0,50.0,2079.0,2959.0,12,9926.0,0.70,-2326.0,5023.0,16,1703.0,880.0,2631.0,-3592.0,0.65
6,P01,LEVEL 3,5500.0,350.0,4200.0,50.0,2181.0,2959.0,12,12015.0,0.74,-3140.0,6599.0,16,2214.0,1363.0,2554.0,-3622.0,0.87
7,P01,LEVEL 2,5500.0,350.0,4200.0,65.0,2755.0,3084.0,12,17226.0,0.89,-4067.0,8671.0,20,4342.0,2005.0,5416.0,-5072.0,0.80
8,P01,LEVEL 1,5500.0,350.0,4200.0,65.0,4655.0,6646.0,20,39367.0,0.70,-6824.0,14585.0,28,16386.0,4650.0,16834.0,-7010.0,0.97
9,P02,LEVEL 9,1200.0,300.0,3000.0,50.0,128.0,616.0,12,778.0,0.21,0.0,124.0,12,55.0,7.0,1361.0,3046.0,0.04


Excel file saved at: C:/Users/zeyad.moustafa/OneDrive - Pritchard Francis/Desktop/Automation/Test Outputs/output.xlsx
Export successful!
